In [13]:
import os   
import librosa
import scipy
import numpy as np
from tqdm import tqdm
import pickle as pkl
import speechpy
import csv
import random
import keras
import pickle as pkl
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier as KNN
# from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error,make_scorer
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [16]:
def parse_data(data_dir):
    all_files = os.listdir(data_dir) 
    features = np.zeros((len(all_files)/2,193)) # feature vector length
    labels = np.zeros(len(all_files)/2)
    ind = 0
    for file in tqdm(all_files):
        if file.endswith(".wav"):
            # mfccs, chroma, mel, contrast,tonnetz = extract_feature(os.path.join(data_dir, file))
            # features[ind] = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
            data_file_name = file[:-4]
            with open(os.path.join(data_dir, data_file_name + "_label.txt")) as l:
                label = l.read()
            if label:
                labels[ind] = 1 #discrepancy
            ind+=1
            # else:
            #     labels = np.append(labels,1) # normal clip
            # label
    return features,labels

def get_normalized_mfcc(mfcc):
    mean = np.mean(mfcc)
    std = np.std(mfcc)
    mfcc_normalized = (mfcc - mean)/std
    return mfcc_normalized

def get_balanced_data(data_dir,pkl_dump_name):
    features = np.zeros((16308*3,46)) # feature vector length
    labels = np.zeros(16308*3)
    ind = 0
    file_names = np.empty((16308*3), dtype = 'object')
    
    for file in tqdm(os.listdir(data_dir)):
        if file.endswith(".wav"):
            data_file_name = file[:-4]
            with open(os.path.join(data_dir, data_file_name + "_label.txt")) as l:
                label = l.read()
            if label:
                labels[ind] = 1 #discrepancy
            mfccs, tonnetz = extract_feature(os.path.join(data_dir, file))
            mfccs_normalized = get_normalized_mfcc(mfccs)
            file_names[ind] = data_file_name
            features[ind] = np.hstack([mfccs_normalized, tonnetz])
            ind += 1

    with open('./data/'+pkl_dump_name+'.pkl','wb') as f:
        pkl.dump((file_names,features,labels),f,protocol=2)
    return features,labels



def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
#     chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
#     mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
#     contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
#     zcr = np.mean(librosa.feature.zero_crossing_rate(y = X).T, axis=1)
    return mfccs,tonnetz


In [17]:
# data = parse_data("./data/TrainingDataUmm")
get_balanced_data("./data/Augmented_Data",'umm_balanced_Mahima_augdata')




  0%|          | 0/97848 [00:00<?, ?it/s]


  0%|          | 2/97848 [00:00<2:33:41, 10.61it/s]


  0%|          | 3/97848 [00:00<3:15:05,  8.36it/s]


  0%|          | 6/97848 [00:00<2:47:34,  9.73it/s]


  0%|          | 7/97848 [00:00<4:26:28,  6.12it/s]


  0%|          | 8/97848 [00:01<4:38:59,  5.84it/s]


  0%|          | 11/97848 [00:01<3:47:04,  7.18it/s]


  0%|          | 13/97848 [00:01<3:53:15,  6.99it/s]


  0%|          | 15/97848 [00:01<3:56:11,  6.90it/s]


  0%|          | 16/97848 [00:02<4:15:45,  6.38it/s]


  0%|          | 18/97848 [00:02<3:44:10,  7.27it/s]


  0%|          | 19/97848 [00:02<5:13:54,  5.19it/s]


  0%|          | 23/97848 [00:02<4:02:26,  6.72it/s]


  0%|          | 25/97848 [00:02<3:37:20,  7.50it/s]


  0%|          | 27/97848 [00:03<4:08:55,  6.55it/s]


  0%|          | 30/97848 [00:03<3:44:10,  7.27it/s]


  0%|          | 33/97848 [00:03<3:07:42,  8.69it/s]


  0%|          | 35/97848 [00:03<2:55:29,  9.29it/s]


  0%|          | 37/978

  1%|          | 521/97848 [01:00<2:52:28,  9.40it/s]


  1%|          | 523/97848 [01:00<3:11:55,  8.45it/s]


  1%|          | 524/97848 [01:00<3:50:49,  7.03it/s]


  1%|          | 526/97848 [01:01<3:28:01,  7.80it/s]


  1%|          | 527/97848 [01:01<3:57:04,  6.84it/s]


  1%|          | 528/97848 [01:01<4:14:20,  6.38it/s]


  1%|          | 531/97848 [01:01<3:45:35,  7.19it/s]


  1%|          | 532/97848 [01:02<5:01:57,  5.37it/s]


  1%|          | 536/97848 [01:02<4:07:25,  6.55it/s]


  1%|          | 537/97848 [01:02<4:20:05,  6.24it/s]


  1%|          | 538/97848 [01:02<5:30:37,  4.91it/s]


  1%|          | 539/97848 [01:03<5:19:11,  5.08it/s]


  1%|          | 540/97848 [01:03<5:09:56,  5.23it/s]


  1%|          | 541/97848 [01:03<5:05:20,  5.31it/s]


  1%|          | 543/97848 [01:03<4:44:59,  5.69it/s]


  1%|          | 545/97848 [01:03<4:31:24,  5.98it/s]


  1%|          | 548/97848 [01:04<3:40:31,  7.35it/s]


  1%|          | 551/97848 [01:04<3:22:50,  7.99

  1%|          | 1049/97848 [02:02<3:53:15,  6.92it/s]


  1%|          | 1050/97848 [02:02<4:10:50,  6.43it/s]


  1%|          | 1051/97848 [02:03<5:23:23,  4.99it/s]


  1%|          | 1052/97848 [02:03<5:18:19,  5.07it/s]


  1%|          | 1053/97848 [02:03<5:12:16,  5.17it/s]


  1%|          | 1054/97848 [02:03<5:07:15,  5.25it/s]


  1%|          | 1058/97848 [02:04<3:58:09,  6.77it/s]


  1%|          | 1071/97848 [02:04<2:53:47,  9.28it/s]


  1%|          | 1074/97848 [02:04<2:31:39, 10.63it/s]


  1%|          | 1077/97848 [02:04<2:47:07,  9.65it/s]


  1%|          | 1083/97848 [02:04<2:12:02, 12.21it/s]


  1%|          | 1086/97848 [02:05<2:21:29, 11.40it/s]


  1%|          | 1088/97848 [02:05<3:04:57,  8.72it/s]


  1%|          | 1090/97848 [02:05<2:56:04,  9.16it/s]


  1%|          | 1092/97848 [02:06<4:25:55,  6.06it/s]


  1%|          | 1095/97848 [02:06<3:36:30,  7.45it/s]


  1%|          | 1100/97848 [02:06<2:49:16,  9.53it/s]


  1%|          | 1102/97848 [02

  2%|▏         | 1628/97848 [03:06<3:02:13,  8.80it/s]


  2%|▏         | 1630/97848 [03:06<4:06:19,  6.51it/s]


  2%|▏         | 1631/97848 [03:06<4:18:59,  6.19it/s]


  2%|▏         | 1632/97848 [03:07<4:32:42,  5.88it/s]


  2%|▏         | 1635/97848 [03:07<3:59:36,  6.69it/s]


  2%|▏         | 1638/97848 [03:07<3:35:53,  7.43it/s]


  2%|▏         | 1639/97848 [03:07<4:02:26,  6.61it/s]


  2%|▏         | 1640/97848 [03:08<4:20:31,  6.15it/s]


  2%|▏         | 1641/97848 [03:08<4:33:02,  5.87it/s]


  2%|▏         | 1642/97848 [03:08<4:46:02,  5.61it/s]


  2%|▏         | 1644/97848 [03:08<4:04:04,  6.57it/s]


  2%|▏         | 1650/97848 [03:08<3:14:56,  8.22it/s]


  2%|▏         | 1652/97848 [03:09<3:28:27,  7.69it/s]


  2%|▏         | 1653/97848 [03:09<3:57:47,  6.74it/s]


  2%|▏         | 1657/97848 [03:09<3:21:54,  7.94it/s]


  2%|▏         | 1659/97848 [03:09<3:30:51,  7.60it/s]


  2%|▏         | 1663/97848 [03:10<2:49:07,  9.48it/s]


  2%|▏         | 1665/97848 [03

  2%|▏         | 2162/97848 [04:06<3:15:08,  8.17it/s]


  2%|▏         | 2166/97848 [04:07<2:38:13, 10.08it/s]


  2%|▏         | 2168/97848 [04:07<2:36:13, 10.21it/s]


  2%|▏         | 2170/97848 [04:07<2:32:43, 10.44it/s]


  2%|▏         | 2174/97848 [04:07<2:22:34, 11.18it/s]


  2%|▏         | 2176/97848 [04:08<3:06:59,  8.53it/s]


  2%|▏         | 2179/97848 [04:08<2:39:24, 10.00it/s]


  2%|▏         | 2181/97848 [04:08<3:02:48,  8.72it/s]


  2%|▏         | 2185/97848 [04:08<2:43:13,  9.77it/s]


  2%|▏         | 2187/97848 [04:09<2:37:36, 10.12it/s]


  2%|▏         | 2189/97848 [04:09<3:01:28,  8.78it/s]


  2%|▏         | 2190/97848 [04:09<3:35:03,  7.41it/s]


  2%|▏         | 2192/97848 [04:09<3:16:15,  8.12it/s]


  2%|▏         | 2193/97848 [04:09<3:51:44,  6.88it/s]


  2%|▏         | 2194/97848 [04:10<4:17:12,  6.20it/s]


  2%|▏         | 2197/97848 [04:10<3:31:17,  7.54it/s]


  2%|▏         | 2201/97848 [04:10<3:06:46,  8.53it/s]


  2%|▏         | 2203/97848 [04

  3%|▎         | 2720/97848 [05:07<2:54:53,  9.07it/s]


  3%|▎         | 2722/97848 [05:08<3:14:50,  8.14it/s]


  3%|▎         | 2724/97848 [05:08<3:47:34,  6.97it/s]


  3%|▎         | 2727/97848 [05:08<3:27:20,  7.65it/s]


  3%|▎         | 2729/97848 [05:09<3:14:57,  8.13it/s]


  3%|▎         | 2730/97848 [05:09<4:35:56,  5.74it/s]


  3%|▎         | 2731/97848 [05:09<4:41:12,  5.64it/s]


  3%|▎         | 2742/97848 [05:09<3:24:55,  7.73it/s]


  3%|▎         | 2745/97848 [05:10<3:25:40,  7.71it/s]


  3%|▎         | 2747/97848 [05:10<4:21:49,  6.05it/s]


  3%|▎         | 2750/97848 [05:10<3:35:49,  7.34it/s]


  3%|▎         | 2755/97848 [05:11<3:03:54,  8.62it/s]


  3%|▎         | 2757/97848 [05:11<3:20:34,  7.90it/s]


  3%|▎         | 2759/97848 [05:11<4:27:56,  5.91it/s]


  3%|▎         | 2761/97848 [05:12<3:53:23,  6.79it/s]


  3%|▎         | 2762/97848 [05:12<4:13:56,  6.24it/s]


  3%|▎         | 2763/97848 [05:12<4:25:39,  5.97it/s]


  3%|▎         | 2764/97848 [05

KeyboardInterrupt: 

In [89]:
with open("./data/umm_balanced_Mahima.pkl","rb") as pk:
    file_names, features, labels = pkl.load(pk)

file_names = file_names.reshape(labels.shape[0], 1)
labels = labels.reshape(labels.shape[0], 1)
# print file_names.shape
# print features.shape
# print labels.shape


x = np.concatenate((file_names, features, labels), axis=1)
# print x
# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.20, random_state = 42)
x = np.random.permutation(x)
orig_x = x
X_train, X_test, y_train, y_test = x[:int(labels.shape[0]*0.7), 1:47], x[int(labels.shape[0]*0.7):, 1:47], x[:int(labels.shape[0]*0.7), 47], x[int(labels.shape[0]*0.7):, 47] 

scaler = StandardScaler()
scaler.fit(X_train)
sc_train = scaler.transform(X_train)
sc_test = scaler.transform(X_test)

In [1]:
# hidden_nodes_list = [300,400,500,600]
# epoch_list = [300,500,700]
# for i in hidden_nodes_list:
#     for j in hidden_nodes_list:
#         for k in hidden_nodes_list:
#             for l in epoch_list:
epochs = 700
n_dim = X_train.shape[1]
n_hidden_units = [300,600,400]

sd = 1 / np.sqrt(n_dim)
learning_rate = 0.002

model = Sequential([
    Dense(n_hidden_units[0], input_shape = (n_dim,), kernel_initializer = keras.initializers.RandomNormal(mean=0, stddev=sd)),
    Activation('tanh'),
    Dense(n_hidden_units[1], kernel_initializer = keras.initializers.RandomNormal(mean=0, stddev=sd)),
    Activation('tanh'),
    Dense(n_hidden_units[2], kernel_initializer = keras.initializers.RandomNormal(mean=0, stddev=sd)),
    Activation('tanh'),
    Dense(1),
    Activation('sigmoid')
])
adam = optimizers.Nadam(lr=learning_rate)# Adam(lr=learning_rate)
model.compile(optimizer = adam, loss = 'binary_crossentropy', metrics=['accuracy'])

model.fit(sc_train,y_train,epochs = epochs, verbose = 1, batch_size = 32, validation_data=(sc_test,y_test))

score = model.evaluate(sc_test,y_test, batch_size = 32)
print (score)

predict = model.predict(sc_test)
y_pred = np.zeros(len(y_test))
y_pred = [1 if p > 0.5 else 0 for p in predict]

count = 0;
false_negatives = []
false_positives = []
for i in range(len(y_test)):
    if y_pred[i] == 0 and y_test[i] == 1:
        false_negatives.append(orig_x[X_train.shape[0] + i][0])
    if y_pred[i] == 1 and y_test[i] == 0:
        false_positives.append(orig_x[X_train.shape[0] + i][0])

print len(false_negatives)
print len(false_positives)

with open("false_negative.txt", "w") as fn:
    for i in range(len(false_negatives)):
        fn.write(str(false_negatives[i] + "\n"))

with open("false_positives.txt", "w") as fp:
    for i in range(len(false_positives)):
        fp.write(str(false_positives[i] + "\n"))
        

#                 model_dict = {"epochs": epochs, "ndim": n_dim, "hidden_units" : ' '.join(str(x) for x in n_hidden_units), "loss" : score[0], "accuracy" : score[1]}

#                 file_check = os.path.isfile("model_iterations.csv")
#                 with open("model_iterations.csv","a+") as csvFile:
#                     if not file_check:
#                         writer.writeheader()
#                     writer = csv.DictWriter(csvFile, model_dict.keys())
#                     writer.writerow(model_dict)


NameError: name 'X_train' is not defined

In [91]:
print false_positives


['segment_651', 'segment_4727', 'segment_1532', 'segment_7826', 'segment_7062', 'segment_7434', 'segment_5211', 'segment_3895', 'segment_7769', 'segment_680', 'segment_5712', 'segment_8312', 'segment_4843', 'segment_5673', 'segment_431', 'segment_3272', 'segment_3392', 'segment_834', 'segment_4875', 'segment_3766', 'segment_3483', 'segment_7687', 'segment_7883', 'segment_6023', 'segment_753', 'segment_7262', 'segment_5401', 'segment_5887', 'segment_7748', 'segment_8146', 'segment_6206', 'segment_6043', 'segment_864', 'segment_1071', 'segment_2940', 'segment_4494', 'segment_2586', 'segment_6668', 'segment_5817', 'segment_7272', 'segment_8234', 'segment_3768', 'segment_2061', 'segment_184', 'segment_3654', 'segment_3057', 'segment_6399', 'segment_6165', 'segment_2138', 'segment_5838', 'segment_7569', 'segment_3361', 'segment_5866', 'segment_6307', 'segment_2555', 'segment_5328', 'segment_2673', 'segment_6955', 'segment_4932', 'segment_1794', 'segment_472', 'segment_6167', 'segment_7841',